In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import h5py
from tqdm.notebook import tqdm_notebook as tqdm
from os import listdir
import seaborn as sns
import time
from scipy.interpolate import interp1d

In [ ]:
def listdir_nohidden(path):
    for f in listdir(path):
        if not f.startswith('.'):
            yield f

In [ ]:
import pandas as pd
from os import listdir

msun = 1.98847e33
folder_path = "/Users/utkarsh/PycharmProjects/fmodes/posterior_fmodes/"
names = list(listdir(folder_path))

# Create empty DataFrames with specified data types
df = pd.DataFrame(columns=["mass", "radius", "fmode"], dtype=float)
df2 = pd.DataFrame(columns=["mass_interp", "radius_interp", "fmode_interp"], dtype=float)

In [ ]:
predM = np.linspace(0, 3*msun, 10000)

for eos_name in tqdm(names):
        path = folder_path + eos_name 
        new = np.genfromtxt(path, delimiter=',', skip_header = 1)
        
        # Delete fmodes higher than limit fmax
        fmax = 3.5e3
        new = new[np.where(new.T[2] < fmax)]
        df = df.append(pd.DataFrame(new, columns=df.columns), ignore_index=True)
        
        M, R, F = new.T

        radius = interp1d(M, R, fill_value= "extrapolate", kind = "cubic")
        fmode = interp1d(M, F, fill_value= "extrapolate", kind = "cubic")

        predR = radius(predM)
        predF = fmode(predM)

        new2 = np.array([predM, predR, predF]).T
        new2 = new2[np.where(new2.T[0] < max(M))]
        new2 = new2[np.where(new2.T[0] > min(M))]
        df2 = df2.append(pd.DataFrame(new2, columns=df2.columns), ignore_index=True)

In [ ]:
upper_list = []
lower_list = []
average_list = []
mass_list = []

for i in tqdm(range(len(predM))):
    curr = df2.loc[df2["mass_interp"] == predM[i]]
    
    if curr.empty:
        continue
    
    curr_R = curr["radius_interp"]
    curr_F = curr["fmode_interp"]
    curr_M = curr["mass_interp"]

    upper = np.percentile(curr_F, 95)
    lower = np.percentile(curr_F, 5)

    upper_list.append(upper)
    lower_list.append(lower)
    average_list.append(np.mean(curr_F))
    mass_list.append(np.mean(curr_M))

average = np.array(average_list)
upper = np.array(upper_list)
lower = np.array(lower_list)
mass = np.array(mass_list)

In [ ]:
plt.figure(dpi = 300)
plt.fill_between(mass/msun, lower/1e3, upper/1e3, alpha=0.1, label = "95%")
plt.plot(mass/msun, average/1e3, label="Average", color="dodgerblue", linewidth=2)
plt.legend(frameon = False)
plt.xlabel("Mass (Msun)")
plt.ylabel("Fundamental Mode (KHz)")
plt.savefig("fmode_envelope.png")
plt.show()

In [ ]:
plt.figure(dpi = 300)
plt.scatter(df.mass/msun, df.fmode/1e3, s = 100, 
            color = "dodgerblue", alpha = 0.01, edgecolor = "white")
plt.xlabel("Mass (Msun)")
plt.ylabel("Fundamental Mode (KHz)")
plt.savefig("fmode_scatter.png")
plt.show()

In [ ]:
plt.figure(dpi = 300)
plt.hist2d(df.mass/msun, df.fmode/1e3, bins=100, cmap = plt.cm.inferno)
plt.xlabel("Mass (MSun)")
plt.ylabel("Fundamental Mode (KHz)")
cbar = plt.colorbar()
cbar.ax.set_ylabel('Counts')
plt.tight_layout()
plt.savefig("fmode_bins.png")
plt.show()

In [ ]:
z = np.polyfit(df.mass/msun, df.fmode/1e3, deg = 3)
p = np.poly1d(z)
x = np.linspace(min(df.mass/msun) + 0.25, max(df.mass/msun) - 0.25, 100)

data_ = np.array([df.mass/msun, df.fmode/1e3]).T
data = pd.DataFrame(data_, columns = ['X','Y'])
plt.figure(figsize=(6, 4), dpi = 300)
sns.kdeplot(data = data, shade=True, x='X', y='Y', cmap = "hot", cut = 0, cbar = True, thresh=0.03,
           bw_adjust=1, levels = 20)
plt.plot(x, p(x),"blue", linewidth = 2, label = "Posterior Mean", linestyle = "dashed")
plt.xlabel("Mass (Msun)")
plt.ylabel("Fundamental Mode (KHz)")
plt.legend(frameon = False)
plt.tight_layout()
plt.savefig("fmode_contour.png")
plt.show()

In [ ]:
plt.figure(dpi = 300)
plt.scatter(df.radius/1e5, df.mass/msun, s = 100, 
            c = df.fmode/1e3, alpha = 0.01, edgecolor = "white",
           cmap = "inferno")
plt.ylabel("Mass (Msun)")
plt.xlabel("Radius (km)")
cbar = plt.colorbar()
cbar.set_alpha(1)
cbar.draw_all()
cbar.set_label('Fundamental Mode (kHz)', rotation=-90, labelpad=15)
plt.savefig("fmode_MR.png")
plt.show()